In [1]:
#Подключаем гугл-диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Устанавливаем PySpark
!pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 45.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=7a4f0de1e2948ece7a17b83f9f548f13b15ad05d1c8c9246aa8975c024a4c5b0
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
#Импортируем библиотеки
import os
import pyspark
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
sc = SparkSession.builder.appName("Big_Data_Lab_2").master("local[*]").getOrCreate()

In [5]:
sc

In [14]:
#Загружаем нужные нам файлы

programming_languages = sc.read.csv("/content/drive/MyDrive/programming-languages.csv")
posts_sample = sc.read.format("xml").options(rowTag="row").load('/content/drive/MyDrive/posts_sample.xml')

In [15]:
posts_sample.take(3)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

In [16]:
#Представим его в виде списка языков программирования
list_of_languages = [str(x[0]) for x in programming_languages.collect()]

In [17]:
#удалим заголовок и посмотрим список
list_of_languages.pop(0)
print(list_of_languages[:25])

['A# .NET', 'A# (Axiom)', 'A-0 System', 'A+', 'A++', 'ABAP', 'ABC', 'ABC ALGOL', 'ABSET', 'ABSYS', 'ACC', 'Accent', 'Ace DASL', 'ACL2', 'ACT-III', 'Action!', 'ActionScript', 'Ada', 'Adenine', 'Agda', 'Agilent VEE', 'Agora', 'AIMMS', 'Alef', 'ALF']


In [18]:
# Зададим функцию, которая будет отметать данные, не относящиеся к периоду с 2010 по 2020 год
def correct_year(x, year):
  start = datetime(year=year, month=1, day=1)
  end = datetime(year=year, month=12, day=31)
  CreationDate = x._CreationDate
  return CreationDate >= start and CreationDate <= end

In [20]:
# Зададим функцию для перевода текста в нижний регистр, нахождения названия языка программирования в тексте и добавления его в кортеж
def finding_language(x):
  tag = None
  for language in list_of_languages:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break
  if tag is None:
    return None
  return (x._Id, tag)

In [21]:
#Задание 1: Сформировать отчёт
programming_languages_report = {}
for year in range(2010, 2020):
  programming_languages_report[year] = posts_sample.rdd\
      .filter(lambda x: x._Tags is not None and correct_year(x, year))\
      .map(finding_language)\
      .filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(
          0,
          lambda x, y: x + 1,
          lambda x1, x2: x1 + x2,
      )\
      .sortBy(lambda x: x[1], ascending=False)\
      .toDF()
  programming_languages_report[year] = programming_languages_report[year].select(col("_1").alias("Programming_language"), 
                                                 col("_2").alias(f"Mentions_in_{year}")).limit(10)

In [22]:
#Выводим отчёт:
for year in range(2010, 2020):
  programming_languages_report[year].show()

+--------------------+----------------+
|Programming_language|Mentions_in_2010|
+--------------------+----------------+
|                Java|              52|
|          JavaScript|              44|
|                 PHP|              42|
|              Python|              25|
|         Objective-C|              22|
|                   C|              20|
|                Ruby|              11|
|              Delphi|               7|
|                   R|               3|
|                Bash|               3|
+--------------------+----------------+

+--------------------+----------------+
|Programming_language|Mentions_in_2011|
+--------------------+----------------+
|                 PHP|              97|
|                Java|              92|
|          JavaScript|              82|
|              Python|              35|
|         Objective-C|              33|
|                   C|              24|
|                Ruby|              17|
|              Delphi|               8|

In [23]:
#Задание 2: Сохраним отчёт
for year in programming_languages_report.keys():
    programming_languages_report[year].write.format("parquet").save(f"/content/drive/MyDrive/programming_language/the_most_popular_in_{year}")